In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import math
from math import floor
import imageio
import json

## Load place

In [2]:
with pd.HDFStore("place.hdf", mode="r") as store:
    print(store.keys())
    place = store.place
    users = store.users

['/place', '/users']


In [3]:
place.describe()

,ts,user_id,x_coordinate,y_coordinate,color
count,1.655990e+07,1.655990e+07,1.655990e+07,1.655990e+07,1.655990e+07
mean,2.139252e+05,2.314466e+05,4.730365e+02,5.154545e+02,5.733436e+00
std,6.689361e+04,2.229644e+05,2.882131e+02,2.996301e+02,4.437919e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.684950e+05,6.805400e+04,2.290000e+02,2.610000e+02,3.000000e+00
50%,2.249860e+05,1.619120e+05,4.590000e+02,5.080000e+02,5.000000e+00
75%,2.657800e+05,3.268200e+05,7.120000e+02,7.870000e+02,9.000000e+00
max,3.200460e+05,1.166923e+06,1.000000e+03,1.000000e+03,1.500000e+01


In [6]:
place = place[place.ts>60845]
place.ts = place.ts - 60845

## Smoothing functions

In [14]:
def get_edits(place, _from, _to):
    canevas = place[(place.ts>=_from)&(place.ts<_to)]\
        .pivot_table(values="ts", index="y_coordinate", columns="x_coordinate", aggfunc=lambda x: len(x))
    canevas = canevas.reindex(index=np.arange(1001), columns=np.arange(1001))
    canevas_matrix = canevas.fillna(0).values[:-1, :-1]
    return canevas_matrix.astype(np.uint8)

In [15]:
def get_gaussian_kernel(kernel_size=3, sigma=2, channels=1):
    # Create a x, y coordinate grid of shape (kernel_size, kernel_size, 2)
    x_coord = torch.arange(kernel_size)
    x_grid = x_coord.repeat(kernel_size).view(kernel_size, kernel_size)
    y_grid = x_grid.t()
    xy_grid = torch.stack([x_grid, y_grid], dim=-1).float()

    mean = (kernel_size - 1)/2.
    variance = sigma**2.

    # Calculate the 2-dimensional gaussian kernel which is
    # the product of two gaussian distributions for two different
    # variables (in this case called x and y)
    gaussian_kernel = (1./(2.*math.pi*variance)) *\
                      torch.exp(
                          -torch.sum((xy_grid - mean)**2., dim=-1) /\
                          (2*variance)
                      )

    # Make sure sum of values in gaussian kernel equals 1.
    gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)

    # Reshape to 2d depthwise convolutional weight
    gaussian_kernel = gaussian_kernel.view(1, 1, kernel_size, kernel_size)
    gaussian_kernel = gaussian_kernel.repeat(channels, 1, 1, 1)

    gaussian_filter = nn.Conv2d(in_channels=channels, out_channels=channels,
                                kernel_size=kernel_size, groups=channels, bias=False)

    gaussian_filter.weight.data = gaussian_kernel
    gaussian_filter.weight.requires_grad = False
    
    return gaussian_filter

In [16]:
def create_canevas_tensors(place, w_size=15*60):
    tss = []
    batch = []
    for ts in range(place.ts.min(), place.ts.max()+w_size, w_size):
        tss.append(ts)
        batch.append(get_edits(place, ts, ts+w_size))
    tensor_batch = torch.cat([torch.Tensor(m).unsqueeze(0) for m in batch])
    tensor_batch.unsqueeze_(1)
    return tss, tensor_batch

In [17]:
def smoothing(image_tensor, kernel, sigma, max_pool):
    image_tensor = image_tensor.cuda(1)
    smoother = nn.Sequential(
        nn.ReflectionPad2d(kernel//2),
        get_gaussian_kernel(kernel, sigma, 1),
        nn.MaxPool2d(max_pool, max_pool)
    ).cuda(1)
    
    smoothed = smoother(image_tensor).cpu().numpy().squeeze(1)
    maxs = F.max_pool2d(image_tensor, image_tensor.size()[-2:], image_tensor.size()[-2:]).cpu().numpy().squeeze()
    return smoothed, maxs

## Apply smoothing globally

In [ ]:
root_path = "levelmaps/"
w_size = 30*60
kernel_size = 3
sigma = 1
strides = 5


In [19]:

tss, canevas_tensor = create_canevas_tensors(place, w_size)

In [20]:
smoothed, maxs = smoothing(canevas_tensor, 3, 1, 5)

In [23]:
for i in range(len(tss)):
    imageio.imwrite(f"levelmaps/global/{i:02}.png", (255*smoothed[i]/smoothed[i].max()).astype(np.uint8), bits=8)

In [24]:
with open("levelmaps/global/index.json", "w") as f:
    json.dump({tss[i]:{"idx":i, "max":int(maxs[i])} for i in range(len(tss))}, f)

## Apply smoothing on biggest "communities"